# FNLP: Lab Session 3

# Hidden Markov Models - Construction and Use



In [1]:
# Import the packages used for this lab

import nltk

# import brown corpus
from nltk.corpus import brown

# module for training a Hidden Markov Model and tagging sequences
from nltk.tag.hmm import HiddenMarkovModelTagger

# module for computing a Conditional Frequency Distribution
from nltk.probability import ConditionalFreqDist

# module for computing a Conditional Probability Distribution
from nltk.probability import ConditionalProbDist

# module for computing a probability distribution with the Maximum Likelihood Estimate
from nltk.probability import MLEProbDist

# pretty printing
import pprint
pp = pprint.PrettyPrinter(indent=4)

#  Corpora tagged with Part-of-Speech information

NLTK provides corpora annotated with Part-of-Speech (POS) information and
some tools to access this information. The Penn Treebank tagset is commonly
used for annotating English sentences. We can inspect this tagset in the following way:

In [2]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

The Brown corpus provided with NLTK is also tagged with POS information,
although the tagset is slightly different than the Penn Treebank tagset. Information about the Brown corpus tagset can be found here:
http://www.scs.leeds.ac.uk/ccalas/tagsets/brown.html

We can retrieve the tagged sentences in the Brown corpus by calling the `tagged_sents()`
function and looking at an annotated sentence:

In [3]:
tagged_sentences = brown.tagged_sents(categories='news')
print('Sentence tagged with Penn Treebank POS labels:')
pp.pprint(tagged_sentences[29])

Sentence tagged with Penn Treebank POS labels:
[   ('The', 'AT'),
    ('jury', 'NN'),
    ('praised', 'VBD'),
    ('the', 'AT'),
    ('administration', 'NN'),
    ('and', 'CC'),
    ('operation', 'NN'),
    ('of', 'IN'),
    ('the', 'AT'),
    ('Atlanta', 'NP-TL'),
    ('Police', 'NNS-TL'),
    ('Department', 'NN-TL'),
    (',', ','),
    ('the', 'AT'),
    ('Fulton', 'NP-TL'),
    ('Tax', 'NN-TL'),
    ("Commissioner's", 'NN$-TL'),
    ('Office', 'NN-TL'),
    (',', ','),
    ('the', 'AT'),
    ('Bellwood', 'NP'),
    ('and', 'CC'),
    ('Alpharetta', 'NP'),
    ('prison', 'NN'),
    ('farms', 'NNS'),
    (',', ','),
    ('Grady', 'NP-TL'),
    ('Hospital', 'NN-TL'),
    ('and', 'CC'),
    ('the', 'AT'),
    ('Fulton', 'NP-TL'),
    ('Health', 'NN-TL'),
    ('Department', 'NN-TL'),
    ('.', '.')]


Sometimes it is useful to use a coarser label set in order to avoid data sparsity
or to allow a mapping between the POS labels for different languages. The Universal tagset was designed to be applicable for all languages:

https://github.com/slavpetrov/universal-pos-tags

There are mappings between the POS tagset of several languages and the Universal tagset. We can access the Universal tags for the Brown corpus sentences
by changing the tagset argument:

In [4]:
tagged_sentences_universal = brown.tagged_sents(categories='news', tagset='universal')
print('Sentence tagged with Universal POS:')
pp.pprint(tagged_sentences_universal[29])

Sentence tagged with Universal POS:
[   ('The', 'DET'),
    ('jury', 'NOUN'),
    ('praised', 'VERB'),
    ('the', 'DET'),
    ('administration', 'NOUN'),
    ('and', 'CONJ'),
    ('operation', 'NOUN'),
    ('of', 'ADP'),
    ('the', 'DET'),
    ('Atlanta', 'NOUN'),
    ('Police', 'NOUN'),
    ('Department', 'NOUN'),
    (',', '.'),
    ('the', 'DET'),
    ('Fulton', 'NOUN'),
    ('Tax', 'NOUN'),
    ("Commissioner's", 'NOUN'),
    ('Office', 'NOUN'),
    (',', '.'),
    ('the', 'DET'),
    ('Bellwood', 'NOUN'),
    ('and', 'CONJ'),
    ('Alpharetta', 'NOUN'),
    ('prison', 'NOUN'),
    ('farms', 'NOUN'),
    (',', '.'),
    ('Grady', 'NOUN'),
    ('Hospital', 'NOUN'),
    ('and', 'CONJ'),
    ('the', 'DET'),
    ('Fulton', 'NOUN'),
    ('Health', 'NOUN'),
    ('Department', 'NOUN'),
    ('.', '.')]


This initial universal tagset was later expanded as part of the Universal Dependencies project. The resulting tagset is called UPOS and you can find more information in the link below. This tagset is not yet supported by NLTK. However, it is important that you know about it since it is the most used multi-lingual tagset nowadays.

https://universaldependencies.org/u/pos/index.html

### Exercise 1: Exploring Brown corpus

In this exercise we will explore the Brown corpus, specifically its frequency distribution over POS tags.
The Brown corpus is divided in topical categories called 'genres'. Let's see what genres we have in the corpus.



In [5]:
pp.pprint(brown.categories())

[   'adventure',
    'belles_lettres',
    'editorial',
    'fiction',
    'government',
    'hobbies',
    'humor',
    'learned',
    'lore',
    'mystery',
    'news',
    'religion',
    'reviews',
    'romance',
    'science_fiction']


You task in this exercise is to implement a function that computes the Frequency Distribution over a Brown genre category and a tagset scheme.
The template of the function is given below. It takes two parameters: one is the genre category and the other is the tagset name.
Your job is to do the following:

1. Convert the list of (word,tag) pairs to a list of tags
2. Use the list of tags to compute a frequency distribution over the tags. Use NLTK's `FreqDist()`
3. Compute the total number of tags in the Frequency Distribution
4. Return the total number of tags and the top 10 most frequent tags

You are given the code to retrieve the list of (word, tag) tuples from the brown corpus corresponding to the given category and tagset.

In [11]:
def explore_tagset_frequency_distro(genre, tagset):
    """Compute a Frequency distribution of the POS tags in a genre of the tagged Brown corpus
    
    :param genre: A Brown corpus genre
    :type genre: str or iterable(str) or None
    :param tagset: A Brown tagset name
    :type tagset: str or None (defaults to 'brown')
    :return: number of tag types, top 10 tags
    :rtype: tuple(int,list(tuple(str,int))"""

    # get the tagged words from the corpus
    tagged_words = brown.tagged_words(categories=genre, tagset=tagset)

    # TODO: convert tagged_words to a list of tags
    tags = ( tp[1] for tp in tagged_words)

    # TODO: using the above list compute a Frequency Distribution
    # hint: use nltk.FreqDist()
    tagsFDist = nltk.FreqDist(tags)

    # TODO: retrieve the number of tag types in the tagset
    # hint: help(nltk.FreqDist)
    number_of_tags = tagsFDist.B()

    # TODO: retrieve the top 10 most frequent tags and their counts
    top_tags = tagsFDist.most_common(10)

    return number_of_tags, top_tags

In [13]:
help(nltk.FreqDist)

Help on class FreqDist in module nltk.probability:

class FreqDist(collections.Counter)
 |  A frequency distribution for the outcomes of an experiment.  A
 |  frequency distribution records the number of times each outcome of
 |  an experiment has occurred.  For example, a frequency distribution
 |  could be used to record the frequency of each word type in a
 |  document.  Formally, a frequency distribution can be defined as a
 |  function mapping from each sample to the number of times that
 |  sample occurred as an outcome.
 |  
 |  Frequency distributions are generally constructed by running a
 |  number of experiments, and incrementing the count for a sample
 |  every time it is an outcome of an experiment.  For example, the
 |  following code will produce a frequency distribution that encodes
 |  how often each word occurs in a text:
 |  
 |      >>> from nltk.tokenize import word_tokenize
 |      >>> from nltk.probability import FreqDist
 |      >>> sent = 'This is an example se

Test your code with this function.


In [12]:
def test_ex1():
    print('Tag FreqDist for news with Penn Treebank tagset:')
    pp.pprint(explore_tagset_frequency_distro('news', None))

    print('Tag FreqDist for science_fiction with Penn Treebank tagset:')
    pp.pprint(explore_tagset_frequency_distro('science_fiction', None))

    # Do the same thing for a different tagset: Universal

    print('Tag FreqDist for news with Universal tagset:')
    pp.pprint(explore_tagset_frequency_distro('news', 'universal'))

    print('Tag FreqDist for science_fiction with Universal tagset:')
    pp.pprint(explore_tagset_frequency_distro('science_fiction', 'universal'))

test_ex1()

Tag FreqDist for news with Penn Treebank tagset:
(   218,
    [   ('NN', 13162),
        ('IN', 10616),
        ('AT', 8893),
        ('NP', 6866),
        (',', 5133),
        ('NNS', 5066),
        ('.', 4452),
        ('JJ', 4392),
        ('CC', 2664),
        ('VBD', 2524)])
Tag FreqDist for science_fiction with Penn Treebank tagset:
(   127,
    [   ('NN', 1541),
        ('IN', 1176),
        ('.', 1077),
        ('AT', 1040),
        (',', 791),
        ('JJ', 723),
        ('NNS', 532),
        ('VBD', 531),
        ('RB', 522),
        ('VB', 495)])
Tag FreqDist for news with Universal tagset:
(   12,
    [   ('NOUN', 30640),
        ('VERB', 14399),
        ('ADP', 12355),
        ('.', 11928),
        ('DET', 11389),
        ('ADJ', 6706),
        ('ADV', 3349),
        ('CONJ', 2717),
        ('PRON', 2535),
        ('PRT', 2264)])
Tag FreqDist for science_fiction with Universal tagset:
(   12,
    [   ('NOUN', 2747),
        ('VERB', 2579),
        ('.', 2428),
        ('D

Let's look at the top tags for different genre and tagsets. Observe differences

# Training and Evaluating an HMM Tagger

NLTK provides a module for training a Hidden Markov Model for sequence tagging.

In [14]:
help(nltk.tag.hmm.HiddenMarkovModelTagger)

Help on class HiddenMarkovModelTagger in module nltk.tag.hmm:

class HiddenMarkovModelTagger(nltk.tag.api.TaggerI)
 |  Hidden Markov model class, a generative model for labelling sequence data.
 |  These models define the joint probability of a sequence of symbols and
 |  their labels (state transitions) as the product of the starting state
 |  probability, the probability of each state transition, and the probability
 |  of each observation being generated from each state. This is described in
 |  more detail in the module documentation.
 |  
 |  This implementation is based on the HMM description in Chapter 8, Huang,
 |  Acero and Hon, Spoken Language Processing and includes an extension for
 |  training shallow HMM parsers or specialized HMMs as in Molina et.
 |  al, 2002.  A specialized HMM modifies training data by applying a
 |  specialization function to create a new training set that is more
 |  appropriate for sequential tagging with an HMM.  A typical use case is
 |  chunking

We can train the HMM for POS tagging given a labelled dataset. At the begging of this lab we learned how to access the labelled sentences of the Brown corpus.
We will use this dataset to study the effect of the size of the training corpus on
the accuracy of the tagger.

### Exercise 2

In this exercise we will train a HMM tagger on a training set and evaluate it
on a test set. The template of the function that you have to implement takes
two parameters: a sentence to be tagged and the size of the training corpus in
number of sentences. You are given the code that creates the training and test
datasets from the tagged sentences in the Brown corpus.

1. Train a Hidden Markov Model tagger on the training dataset. Refer to `help(nltk.tag.hmm.HiddenMarkovModelTagger.train)` if necessary.
2. Use the trained model to tag the sentence
3. Use the trained model to evaluate the tagger on the test dataset

In [15]:
def my_trainer(sentence, size):
    """ Create an HMM tagger from the Brown news corpus, and test it by tagging a sample sentence
    
    :param sentence: An untagged sentence as an example
    :type sentence: list(str)
    :param size: Number of sentences to train on (be sure to leave room for the test data)
    :type size: int
    :return: The tagger, the sample sentence with tags, entropy of model wrt 100 test sentences
    :rtype: tuple(nltk.tag.hmm.HiddenMarkovModelTagger, list(tuple(str,str)), float)"""
    tagged_sentences = brown.tagged_sents(categories='news')

    # set up the training data
    train_data = tagged_sentences[-size:]

    # set up the test data
    test_data = tagged_sentences[:100]
    
    # Hint: use help on HiddenMarkovModelTagger to find out how to train, tag and evaluate an HMM tagger

    # TODO: train a HiddenMarkovModelTagger, using the train() method
    tagger = HiddenMarkovModelTagger.train(train_data)

    # TODO: using the hmm tagger tag the sentence
    hmm_tagged_sentence = tagger.tag(sentence)

    # TODO: using the hmm tagger, evaluate accuracy score on the test data
    acc = tagger.evaluate(test_data)

    return tagger, hmm_tagged_sentence, acc

Test your implementation by running:

In [17]:
def test_my_trainer():
    tagged_sentences = brown.tagged_sents(categories='news')
    words = [tp[0] for tp in tagged_sentences[42]]
    tagger, hmm_tagged_sentence, acc = my_trainer(words, 500)
    print('Training nltk.HiddenMarkovModelTagge with 500 sentences...')
    print('\tSentence tagged with model:')
    pp.pprint(hmm_tagged_sentence)
    print('\tAccuracy score on the test set: %.4f%%' % (100.0*acc))
    print()

    tagger, hmm_tagged_sentence, acc = my_trainer(words, 3000)
    print('Training nltk.HiddenMarkovModelTagge with 3000 sentences...')
    print('\tSentence tagged with model:')
    pp.pprint(hmm_tagged_sentence)
    print('\tAccuracy score on the test set: %.4f%%' % (100.0*acc))

test_my_trainer()

Training nltk.HiddenMarkovModelTagge with 500 sentences...
	Sentence tagged with model:
[   ('The', 'AT'),
    ("mayor's", 'QL'),
    ('present', 'JJ'),
    ('term', 'NN'),
    ('of', 'IN'),
    ('office', 'NN'),
    ('expires', '.'),
    ('Jan.', '.'),
    ('1', '.'),
    ('.', '.')]
	Accuracy score on the test set: 71.4727%

Training nltk.HiddenMarkovModelTagge with 3000 sentences...
	Sentence tagged with model:
[   ('The', 'AT'),
    ("mayor's", 'NN$'),
    ('present', 'JJ'),
    ('term', 'NN'),
    ('of', 'IN'),
    ('office', 'NN'),
    ('expires', 'IN'),
    ('Jan.', 'NP'),
    ('1', 'CD'),
    ('.', '.')]
	Accuracy score on the test set: 86.8607%


Look at the tagged sentence and the accuracy of the tagger. How does the size of the training set affect the accuracy?


# Computing the Transition and Emission Probabilities

In the previous exercise we learned how to train and evaluate an HMM tagger.
We have used the HMM tagger as a black box and have seen how the training
data affects the accuracy of the tagger. In order to get a better understanding
of the HMM we will look at the two components of this model:
    
* The transition model
* The emission model

The transition model estimates $P (tag_{i+1} |tag_i )$, the probability of a POS tag
at position $i+1$ given the previous tag (at position $i$). The emission model
estimates $P (word|tag)$, the probability of the observed word given a tag.

Given the above definitions, we will need to learn a Conditional Probability
Distribution for each of the models.

In [18]:
help(nltk.probability.ConditionalProbDist)

Help on class ConditionalProbDist in module nltk.probability:

class ConditionalProbDist(ConditionalProbDistI)
 |  A conditional probability distribution modeling the experiments
 |  that were used to generate a conditional frequency distribution.
 |  A ConditionalProbDist is constructed from a
 |  ``ConditionalFreqDist`` and a ``ProbDist`` factory:
 |  
 |  - The ``ConditionalFreqDist`` specifies the frequency
 |    distribution for each condition.
 |  - The ``ProbDist`` factory is a function that takes a
 |    condition's frequency distribution, and returns its
 |    probability distribution.  A ``ProbDist`` class's name (such as
 |    ``MLEProbDist`` or ``HeldoutProbDist``) can be used to specify
 |    that class's constructor.
 |  
 |  The first argument to the ``ProbDist`` factory is the frequency
 |  distribution that it should model; and the remaining arguments are
 |  specified by the ``factory_args`` parameter to the
 |  ``ConditionalProbDist`` constructor.  For example, the f

### Exercise 3: Emission Model

In this exercise we will estimate the emission model. In order to compute the
Conditional Probability Distribution of $P (word|tag)$ we first have to compute
the Conditional Frequency Distribution of a word given a tag.



In [19]:
help(nltk.probability.ConditionalFreqDist)

Help on class ConditionalFreqDist in module nltk.probability:

class ConditionalFreqDist(collections.defaultdict)
 |  A collection of frequency distributions for a single experiment
 |  run under different conditions.  Conditional frequency
 |  distributions are used to record the number of times each sample
 |  occurred, given the condition under which the experiment was run.
 |  For example, a conditional frequency distribution could be used to
 |  record the frequency of each word (type) in a document, given its
 |  length.  Formally, a conditional frequency distribution can be
 |  defined as a function that maps from each condition to the
 |  FreqDist for the experiment under that condition.
 |  
 |  Conditional frequency distributions are typically constructed by
 |  repeatedly running an experiment under a variety of conditions,
 |  and incrementing the sample outcome counts for the appropriate
 |  conditions.  For example, the following code will produce a
 |  conditional freque

The constructor of the ConditionalFreqDist class takes as input a list of tuples,
each tuple consisting of a condition and an observation. For the emission model,
the conditions are tags and the observations are the words. The template of the
function that you have to implement takes as argument the list of tagged words
from the Brown corpus.

1. Build the dataset to be passed to the `ConditionalFreqDist()` constructor. Words should be lowercased. Each item of data should be a tuple of tag (a condition) and word (an observation).
2. Compute the Conditional Frequency Distribution of words given tags.
3. Return the top 10 most frequent words given the tag NN.
4. Compute the Conditional Probability Distribution for the above Conditional Frequency Distribution. Use the `MLEProbDist` estimator when calling the ConditionalProbDist constructor.
5. Compute the probabilities:

 $P(\text{year}|\text{NN})$ 
 
 $P(\text{year}|\text{DT})$

In [20]:
def my_emission_model(tagged_words):
    """Build and sample Conditional{Freq->Prob}Dist for word given tag from a list of tagged words using MLE
    
    :param tagged_words: tagged words (word,tag)
    :type tagged_words: list(tuple(str,str))
    :return: Conditional Freq dist of word given tag, top 10 words with tag NN,
             Conditional Prob dist of word given tag, P('year'|'NN'), P('year'|'DT')
    :rtype: tuple(nltk.probability.ConditionalFreqDist,list(tuple(str,int)),nltk.probability.ConditionalProbDist,float,float)"""
    
    # in the previous labs we've seen how to build a freq dist
    # we need conditional distributions to estimate the transition and emission models
    # in this exercise we estimate the emission model
    
    # TODO: prepare the data
    # the data object should be a list of tuples of conditions and observations
    # in our case the tuples should be of the form (tag,word) where words are lowercased
    data = [ (tag, word.lower()) for (word, tag) in tagged_words]

    # TODO: compute a Conditional Frequency Distribution for words given their tags using our data
    emission_FD =ConditionalFreqDist(data)

    # TODO: find the top 10 most frequent words given the tag NN
    top_NN = emission_FD['NN'].most_common(10)

    # TODO: Compute the Conditional Probability Distribution using the above Conditional Frequency Distribution. 
    #       Use nltk.probability.MLEProbDist estimator.
    emission_PD =ConditionalProbDist(emission_FD,MLEProbDist)

    # TODO: compute the probabilities of P(year|NN) and P(year|DT)
    p_NN = emission_PD['NN'].prob('year')
    p_DT = emission_PD['DT'].prob('year')

    return emission_FD, top_NN, emission_PD, p_NN, p_DT

Test your implementation by running:

In [21]:
def test_emission_model():
    tagged_words = brown.tagged_words(categories='news')
    (emission_FD, top_NN, emission_PD, p_NN, p_DT) = my_emission_model(tagged_words)
    print('Frequency of words given the tag *NN*: ')
    pp.pprint(top_NN)
    print('P(year|NN) = ', p_NN)
    print('P(year|DT) = ', p_DT)

test_emission_model()

Frequency of words given the tag *NN*: 
[   ('year', 137),
    ('time', 98),
    ('state', 92),
    ('week', 86),
    ('man', 72),
    ('home', 72),
    ('program', 65),
    ('school', 65),
    ('night', 64),
    ('day', 62)]
P(year|NN) =  0.0104087524692296
P(year|DT) =  0.0


Look at the estimated probabilities. Why is P(year|DT) = 0 ? 

What are the problems with having zero (0) probabilities and what can be done to
avoid this?

### Exercise 4: Transition Model

In this exercise we will estimate the transition model. In order to compute the
Conditional Probability Distribution of $P (tag_{i+1} |tag_i )$ we first have to compute
the Conditional Frequency Distribution of a tag at position $i + 1$ given the previous tag.

The constructor of the `ConditionalFreqDist` class takes as input a list of tuples, each tuple consisting of a condition and an observation. For the transition
model, the conditions are tags at position i and the observations are tags at
position $i + 1$. The template of the function that you have to implement takes
as argument the list of tagged sentences from the Brown corpus.

1. Build the dataset to be passed to the `ConditionalFreqDist()` constructor. Each item in your data should be a pair of condition and observation: $(tag_i,tag_{i+1})$
2. Compute the Conditional Frequency Distribution of a tag at position $i + 1$ given the previous tag.
3. Compute the Conditional Probability Distribution for the above Conditional Frequency Distribution. Use the `MLEProbDist` estimator when calling the `ConditionalProbDist` constructor.
4. Compute the probabilities 
   
   $P(\text{NN}|\text{VBD})$ 
   
   $P(\text{NN}|\text{DT})$

In [ ]:
def my_transition_model(tagged_sentences):
    """Build and sample Conditional{Freq->Prob}Dist for tag given preceding tag from a list of tagged words using MLE
    
    :param tagged_sentences: Tagged sentences for training and testing
    :type tagged_sentences: list(list(tuple(str,str)))
    :return: Conditional Freq dist of tag given preceding tag,
             Conditional Prob dist of tag given preceding tag, P('NN'|'VBD') and P('NN'|'DT')
    :rtype: tuple(nltk.probability.ConditionalFreqDist,nltk.probability.ConditionalProbDist,float,float)"""
    
    # TODO: prepare the data
    # the data object should be an array of tuples of conditions and observations
    # in our case the tuples will be of the form (tag_(i),tag_(i+1))
    tagGenerators=(((s[i][1],s[i+1][1]) for i in range(len(s)-1)) for s in tagged_sentences)
    # tagGenerators is an iterator of iterators of pairs of tags
    # The following chains them all together to produce an iterator of pairs of tags
    data = itertools.chain.from_iterable(tagGenerators)

    # TODO: compute a Conditional Frequency Distribution for a tag given the previous tag
    transition_FD =ConditionalFreqDist(data)

    # TODO: compute a Conditional Probability Distribution for the
    # transition probability P(tag_(i+1)|tag_(i)) using an MLEProbDist
    # to estimate the probabilities
    transition_PD =ConditionalProbDist(transition_FD, MLEProbDist)

    # TODO: compute the probabilities of P('NN'|'VBD') and P('NN'|'DT')
    p_VBD_NN = transition_PD['VBD'].prob('NN')
    p_DT_NN = transition_PD['DT'].prob('NN')

    return transition_FD, transition_PD, p_VBD_NN, p_DT_NN


Test your implementation by running:

In [ ]:
def test_transition_model():
    tagged_sentences = brown.tagged_sents(categories='news')
    (transition_FD, transition_PD, p_VBD_NN, p_DT_NN) = my_transition_model(tagged_sentences)
    print('P(NN|VBD) = ', p_VBD_NN)
    print('P(NN|DT) = ', p_DT_NN)
    
test_transition_model()

Are the results what you would expect? The sequence NN DT seems very probable. 

How will this affect the sequence tagging?

# Going further

Modify your code for exercise 3 to use a different estimator, to introduce some
smoothing, and compare the results with the original.
In exercise 4 we didn’t do anything about the boundaries. Modify your code for
exercise 4 to use `<s>` at the beginning of every sentence and `</s>` at the end.

Explore the resulting conditional probabilities. What is the most likely tag at
the beginning of a sentence? At the end?